attempt of making a calibration plot

In [5]:
import sys

import pandas as pd
import numpy as np
import json
import os

import torch
import ml_insights as mli
import matplotlib.pyplot as plt

In [6]:
sys.path.append('../')
import datasets
import models
import utils


In [7]:
HIGH_RES = True
THRESHOLD = 0.5
DISABLE_OCEAN_MASK = False
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SET_MAX_CMAP_TO_1 = False

In [8]:
# load model
train_params = torch.load('../pretrained_models/model_an_full_input_enc_sin_cos_hard_cap_num_per_class_1000.pt', map_location='cpu')
model = models.get_model(train_params['params'])
model.load_state_dict(train_params['state_dict'], strict=True)
model = model.to(DEVICE)
model.eval()

ResidualFCNet(
  (class_emb): Linear(in_features=256, out_features=47375, bias=False)
  (feats): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=256, bias=True)
    )
    (4): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=

In [9]:
#load reference from iucn
with open(os.path.join('../data/eval/iucn/', 'iucn_res_5.json'), 'r') as f:
            data = json.load(f)

In [10]:
species_ids = list((data['taxa_presence'].keys()))

In [20]:
len(species_ids)

2418

In [11]:
if train_params['params']['input_enc'] in ['env', 'sin_cos_env']:
    raster = datasets.load_env()
else:
    raster = None
enc = utils.CoordEncoder(train_params['params']['input_enc'], raster=raster)

In [12]:
obs_locs = np.array(data['locs'], dtype=np.float32)
obs_locs = torch.from_numpy(obs_locs).to('cpu')
loc_feat = enc.encode(obs_locs)

In [13]:
classes_of_interest = torch.zeros(len(species_ids), dtype=torch.int64)
taxa_ids = torch.zeros(len(species_ids), dtype=torch.int64)
for tt_id, tt in enumerate(species_ids):
    class_of_interest = np.array([train_params['params']['class_to_taxa'].index(int(tt))])
    classes_of_interest[tt_id] = torch.from_numpy(class_of_interest)
    taxa_ids[tt_id] = int(tt)

In [14]:
with torch.no_grad():
    loc_emb = model(loc_feat, return_feats=True)
    wt = model.class_emb.weight[classes_of_interest, :]

In [24]:
def create_calibration_curves(taxa, preds, species_locs):
    truth_array = np.zeros(preds.shape, int)
    truth_array[species_locs] = 1
    
    fig, ax = plt.subplots(figsize=(10,5))
    fig.suptitle("taxa "+ str(taxa), y=0.9)
    rd = mli.plot_reliability_diagram(truth_array, preds, marker='.', show_histogram=True)
    #plt.savefig(f'../calibration_plots/calibration_curves/{taxa}.png')
    return rd


In [25]:
import geopandas as gpd
df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

def plot_map(taxa,preds,species_locs):
    test = np.array(data['locs'])
    plot_array = test[species_locs]
    thresholds = [0.02,0.1,0.5]

    mask1 = preds>thresholds[0]
    plot_threhs1 = test[mask1]

    mask2 = preds>thresholds[1]
    plot_threhs2 = test[mask2]

    mask3 = preds>thresholds[2]
    plot_threhs3 = test[mask3]

    fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(10,5))
    fig.suptitle("taxa " + str(taxa),  y=0.75)
    #iucn map
    df.plot(ax=ax1,color="lightgray")
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.scatter(plot_array[:,0], plot_array[:,1], color='firebrick', s=10, alpha=0.05)

    #predictions

    df.plot(ax=ax2,color="lightgray")

    ax2.set_xticks([])
    ax2.set_yticks([])

    ax2.scatter(plot_threhs1[:,0], plot_threhs1[:,1], color='lightsteelblue', s=10, alpha=0.05)
    ax2.scatter(plot_threhs2[:,0], plot_threhs2[:,1], color='cornflowerblue', s=10, alpha=0.05)
    ax2.scatter(plot_threhs3[:,0], plot_threhs3[:,1], color='mediumblue', s=10, alpha=0.05)

    plt.tight_layout()
    #plt.savefig(f'../calibration_plots/maps/{taxa}map.png')

/var/folders/pp/c6m9_85s5rd16hbzmflk4b9r0000gn/T/ipykernel_20486/4244665464.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [26]:
species_dict = dict()
for tt_id, taxa in enumerate(taxa_ids):
    wt_1 = wt[tt_id,:]
    preds = torch.sigmoid(torch.matmul(loc_emb, wt_1)).cpu().numpy()
    taxa = taxa.item()
    species_locs = data['taxa_presence'].get(str(taxa))
    rd = create_calibration_curves(taxa,preds,species_locs)
    species_dict[taxa] = rd

    plot_map(taxa,preds,species_locs);

0
1
2
3
4
5
6
7
8
9
10


/var/folders/pp/c6m9_85s5rd16hbzmflk4b9r0000gn/T/ipykernel_20486/1698052475.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(10,5))


11
12
13
14
15


Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7f8cdc1f7310>
Traceback (most recent call last):
  File "/Users/filipdorm/opt/anaconda3/envs/sinr_icml/lib/python3.9/site-packages/matplotlib/transforms.py", line 209, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id(self): pop(k))
KeyboardInterrupt: 


16
17
18
19
20
21
22
23
24
25
26
27


In [23]:
"""import pickle
with open('species_calibration_curves.pkl', 'wb') as fp:
    pickle.dump(species_dict, fp)
    print('dictionary saved successfully to file')"""

dictionary saved successfully to file
